In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# If you need fully deterministic results between runs, set the following environment value prior to launching jupyter.
# See comment in sherlock.features.paragraph_vectors.infer_paragraph_embeddings_features for more info.
%env PYTHONHASHSEED

'13'

# Extract features, retrain Sherlock and generate predictions.

The script below first downloads the data (roughly 700K samples), then extract features from the raw data values. <br>

In [3]:
from datetime import datetime
import os
import sys
import time

import numpy as np
import pandas as pd

from sherlock import helpers
from sherlock.functional import extract_features_to_csv
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (
    extract_features,
    convert_string_lists_to_lists,
    prepare_feature_extraction,
    load_parquet_values,
)
from sherlock.features.word_embeddings import initialise_word_embeddings

In [4]:
print(f'Started at {datetime.now()}.')

Started at 2022-02-11 08:55:34.107148.


## Download data
This will download the raw values and preprocessed files, the corresponding labels as well as a few other supporting files:
- `download_data()` will download 3.6GB of data for preprocessing and model training into the `data/` directory.
- `prepare_feature_extraction()` will download +/- 800 MB of data into the `features/` directory.

In [5]:
helpers.download_data()

Data was downloaded.


In [6]:
if not os.path.exists('../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy'):
    raise SystemExit(
        """
        Trained paragraph vectors do not exist,
        please run the '01-train-paragraph-vector-features' notebook before continuing
        """
    )

## Read in raw data
You can skip this step if you want to use a preprocessed data file.

## Report memory usage (can be slow)

In [7]:
report_memory = False

## Extract features

The input data is assumed to be a dataframe of column values stored as stringed lists "['hello', 'goodbye', 'hi']", stored in a parquet file.

`TODO`: ideally we pickle the dataframe with value lists to avoid parsing these strings into lists again.

In [8]:
timestr = time.strftime("%Y%m%d-%H%M%S")

# Features will be output to the following files
X_test_filename_csv = f'../data/data/processed/test_{timestr}.csv'
X_train_filename_csv = f'../data/data/processed/train_{timestr}.csv'
X_validation_filename_csv = f'../data/data/processed/validation_{timestr}.csv'

### PREPARATION

In [9]:
# ensure embedding initialisation is outside of timing for extract_features
prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 2 files:
        
 ../sherlock/features/glove.6B.50d.txt and 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Word Embeddings process took 0:00:16.095174 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:09.854615 seconds. (filename = ../sherlock/features/par_vec_trained_400.pkl)
Initialised NLTK, process took 0:00:00.342873 seconds.


[nltk_data] Downloading package punkt to /Users/madelon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/madelon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
%load_ext line_profiler

In [11]:
# default process creation mode is changed in Python 3.8 to 'spawn' which causes "name not defined" type errors. Force fork 'mode' for now (this only needs to be called once).
# https://bugs.python.org/issue39931
#mp.set_start_method('fork', force=True)

## EXTRACT FEATURES TO CSV

### TEST SET

In [12]:
values = load_parquet_values("../data/data/raw/test_values.parquet")

extract_features_to_csv(X_test_filename_csv, values)

values = None

Starting ../data/data/processed/test_20220211-085607.csv at 2022-02-11 08:56:51.242296. Rows=137353, using 8 CPU cores
[
  [
    "['Central Missouri', 'unattached', 'unattached', 'Kansas State University', 'unattached', 'North Dakota State', 'Nike']",
    "[95, 100, 95, 89, 84, 91, 88, 94, 75, 78, 90, 84, 90, 76, 93, 70, 80, 80, 82]",
    "['Katie Crews', 'Christian Hiraldo', 'Alex Estrada', 'Fredy Peltroche', 'Xavier Perez', 'Gustavo Larrosa', 'Jose Montano', 'Angel Cruz (7)', 'J Acosta']",
    "['Christian', 'Non-Christian', 'Unreported', 'Jewish', 'Atheists']",
    "['AAF-McQuay Canada Inc.', 'AAF-McQuay Canada Inc.', 'Ability Janitorial Services Ltd.', 'Acart Communications Inc.', 'Accu-lift Flooring Systems', 'Accurate Point Construction Ltd.  ', 'Acklands-Grainger Inc.', 'Acme Future Security Controls Inc.', 'ACMG Management Inc.', 'Advanced Chippers Technologies Inc.', 'Advanced Chippewa Technologies Inc.', 'Advanced Chippewa Technologies Inc.', 'Advanced Chippewa Technologies I

['n_[0]-agg-any', 'n_[0]-agg-all', 'n_[0]-agg-mean', 'n_[0]-agg-var', 'n_[0]-agg-min', 'n_[0]-agg-max', 'n_[0]-agg-median', 'n_[0]-agg-sum', 'n_[0]-agg-kurtosis', 'n_[0]-agg-skewness', 'n_[1]-agg-any', 'n_[1]-agg-all', 'n_[1]-agg-mean', 'n_[1]-agg-var', 'n_[1]-agg-min', 'n_[1]-agg-max', 'n_[1]-agg-median', 'n_[1]-agg-sum', 'n_[1]-agg-kurtosis', 'n_[1]-agg-skewness', 'n_[2]-agg-any', 'n_[2]-agg-all', 'n_[2]-agg-mean', 'n_[2]-agg-var', 'n_[2]-agg-min', 'n_[2]-agg-max', 'n_[2]-agg-median', 'n_[2]-agg-sum', 'n_[2]-agg-kurtosis', 'n_[2]-agg-skewness', 'n_[3]-agg-any', 'n_[3]-agg-all', 'n_[3]-agg-mean', 'n_[3]-agg-var', 'n_[3]-agg-min', 'n_[3]-agg-max', 'n_[3]-agg-median', 'n_[3]-agg-sum', 'n_[3]-agg-kurtosis', 'n_[3]-agg-skewness', 'n_[4]-agg-any', 'n_[4]-agg-all', 'n_[4]-agg-mean', 'n_[4]-agg-var', 'n_[4]-agg-min', 'n_[4]-agg-max', 'n_[4]-agg-median', 'n_[4]-agg-sum', 'n_[4]-agg-kurtosis', 'n_[4]-agg-skewness', 'n_[5]-agg-any', 'n_[5]-agg-all', 'n_[5]-agg-mean', 'n_[5]-agg-var', 'n_[5]-agg

Finished. Processed 137353 rows in 0:11:28.580070, key_count=8


In [13]:
print(f'Finished at {datetime.now()}')

Finished at 2022-02-11 09:08:20.271759


### TRAIN SET

In [13]:
values = load_parquet_values("../data/data/raw/train_values.parquet")

extract_features_to_csv(X_train_filename_csv, values)

values = None

In [21]:
print(f'Finished at {datetime.now()}')

Finished at 2022-02-10 18:28:31.533353


### VALIDATION SET

In [15]:
values = load_parquet_values("../data/data/raw/val_values.parquet")

extract_features_to_csv(X_validation_filename_csv, values)

values = None

In [23]:
print(f'Finished at {datetime.now()}')

Finished at 2022-02-10 18:33:02.388902


### Read Locally Processed Features

In [24]:
start = datetime.now()

X_test = pd.read_csv(X_test_filename_csv, dtype=np.float32)

print(f'Load Features (test) process took {datetime.now() - start} seconds.')

Load Features (test) process took 0:00:31.909566 seconds.


In [25]:
X_test.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.124150,-0.160029,-0.140256,0.109107,-0.044414,-0.124615,-0.033864,0.111789,-0.014209,-0.160459
1,1.0,0.0,0.368421,0.337950,0.0,2.0,0.0,7.0,0.742677,1.326868,...,-0.060843,-0.009223,-0.062848,0.080908,-0.240975,-0.039260,-0.050986,-0.068109,-0.063214,-0.200062
2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.254407,-0.140454,-0.077885,0.270047,-0.254806,-0.015952,-0.358635,-0.090083,0.120246,0.006599
3,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.096018,0.018162,-0.066431,-0.039642,-0.104596,-0.087560,-0.108518,0.023666,0.017810,-0.047467
4,1.0,0.0,0.020080,0.035741,0.0,2.0,0.0,5.0,96.521561,9.784149,...,-0.785815,-0.388175,-1.259390,1.080410,-0.445470,-0.532897,0.137333,-1.173960,1.222390,-1.423210


In [26]:
start = datetime.now()

X_train = pd.read_csv(X_train_filename_csv, dtype=np.float32)

print(f'Load Features (train) process took {datetime.now() - start} seconds.')

Load Features (train) process took 0:01:40.003595 seconds.


In [27]:
X_train.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.047276,-0.050462,-0.131864,0.068065,-0.074033,0.016362,0.055275,-0.063609,-0.014739,-0.076884
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.169210,-0.091240,-0.008702,0.158941,-0.102764,-0.001776,0.035398,0.005877,-0.033673,-0.266011
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,0.086971,0.152903,-0.141693,0.364022,-0.708731,0.064241,-0.183656,-0.001072,0.235746,-0.530483
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,0.019576,-0.020715,0.080360,0.170148,-0.339541,-0.025463,0.062125,-0.004156,-0.044532,-0.101253
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.264917,-0.144005,-0.086391,0.027880,-0.394154,-0.057364,-0.211917,0.069740,0.095384,-0.356454


In [28]:
start = datetime.now()

X_validation = pd.read_csv(X_validation_filename_csv, dtype=np.float32)

print(f'Load Features (validation) process took {datetime.now() - start} seconds.')

Load Features (validation) process took 0:00:30.514030 seconds.


In [29]:
X_validation.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.105711,0.181456,-0.294050,0.163736,-0.496933,0.069320,-0.040903,-0.056309,0.334068,-0.391248
1,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.258922,-0.001249,-0.231181,0.288208,-0.181530,0.052967,-0.037158,-0.016913,0.032424,-0.311296
2,1.0,0.0,0.25000,0.187500,0.0,1.0,0.0,10.0,-0.666667,1.154701,...,-0.265864,-0.388315,-0.133144,-0.209464,-0.042673,-0.084304,-0.112698,0.147051,0.224680,-0.373928
3,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.000000,...,-0.198478,0.041071,-0.265879,0.261911,-0.264352,-0.115126,-0.271292,-0.061009,0.032532,-0.105325
4,1.0,0.0,0.09434,0.311855,0.0,4.0,0.0,5.0,42.275021,6.530585,...,-0.199170,-0.486600,0.026245,-0.288035,-0.301814,-0.007751,0.102981,0.405819,0.632662,-0.274999


## Impute NaN values with feature means

In [30]:
start = datetime.now()

train_columns_means = pd.DataFrame(X_train.mean()).transpose()

print(f'Transpose process took {datetime.now() - start} seconds.')

Transpose process took 0:00:11.975168 seconds.


In [31]:
start = datetime.now()

X_train.fillna(train_columns_means.iloc[0], inplace=True)
X_validation.fillna(train_columns_means.iloc[0], inplace=True)
X_test.fillna(train_columns_means.iloc[0], inplace=True)

train_columns_means=None

print(f'FillNA process took {datetime.now() - start} seconds.')

FillNA process took 0:00:04.992048 seconds.


In [32]:
start = datetime.now()

X_train.to_parquet('../data/data/processed/train.parquet', engine='pyarrow', compression='snappy')
X_validation.to_parquet('../data/data/processed/validation.parquet', engine='pyarrow', compression='snappy')
X_test.to_parquet('../data/data/processed/test.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process took {datetime.now() - start} seconds.')

Save parquet process took 0:01:03.475468 seconds.


In [33]:
print(f'Completed at {datetime.now()}.')

Completed at 2022-02-10 18:37:05.949284.
